# ChIPSeq Pipeline

## Be sure to install paramiko and scp with pip before using this notebook

## 1. Configure AWS key pair, data location on S3 and the project information

This cell only contains information that you, the user, should input.

#### String Fields

**s3_input_files_address**: This is an s3 path to where your input fastq files are found. This shouldn't be the path to the actual fastq files, just to the directory containing all of them. All fastq files must be in the same s3 bucket.

**s3_output_files_address**: This is an s3 path to where you would like the outputs from your project to be uploaded. This will only be the root directory, please see the README for information about exactly how outputs are structured

**design_file**: This is a path to your design file for this project. Please see the README for the format specification for the design files. 

**your_cluster_name**: This is the name given to your cluster when it was created using cfncluster. 

**private_key**: The path to your private key needed to access your cluster.

**project_name**: Name of your project. There should be no whitespace.

**workflow**: The workflow you want to run for this project. For the ChIPSeq pipeline the only possible workflow is "homer".

**genome**: The name of the reference you want to use for your project. Currently only "hg19" and "mm10" are supported here.

**style**: This will always be either "factor" or "histone" depending on your purposes. The "factor" style is more fitting for transcription factor analysis while the "histone" style is intended for histone analysis. More details can be found [here](http://homer.ucsd.edu/homer/ngs/peaks.html)

#### analysis_steps
This is a set of strings that contains the steps you would like to run. The order of the steps does not matter.

posible homer steps: "fastqc", "trim", "align", "multiqc", "make_tag_directory", "make_UCSC_file", "find_peaks", "annotate_peaks", "pos2bed", "find_motifs_genome"

In [4]:
import os
import sys
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager
from util import PipelineManager
from util import DesignFileLoader

#s3 addresses for input files and output files
s3_input_files_address = "s3://ucsd-ccbb-interns/Mustafa/chip_test/"
s3_output_files_address = "s3://ucsd-ccbb-interns/Mustafa/chip_test/"

## CFNCluster name
your_cluster_name = "mustafa7"

## The private key pair for accessing cluster.
private_key = "/home/mustafa/keys/interns_oregon_key.pem"

## Project information
project_name = "histone_output_check"

#options: homer
workflow = "homer"

#options: hg19, mm10
genome = "hg19"

#options: factor, histone
style = "histone"

## If delete cfncluster after job is done.
delete_cfncluster= False


##order does not matter

##can be fastqc, trim, bowtie, multiqc, make_tag_directory, make_UCSC_file, 
#find_peaks, annotate_peaks, pos2bed, find_motifs_genome
analysis_steps = {
                    "fastqc"
                    ,"trim"
                    ,"align"
#                     ,"multiqc"
                    ,"make_tag_directory"
                    ,"make_UCSC_file"
                    ,"find_peaks"
                    ,"annotate_peaks"
                    ,"pos2bed"
                    ,"find_motifs_genome"
                }

#path to chipseq design file
#examples in cirrus_root/data/cirrus-ngs/
design_file = "/home/mustafa/ccbb/cirrus-ngs/data/cirrus-ngs/testing_chip.txt"

print("variables set")

variables set


## 2. Create CFNCluster

Following cell connects to your cluster. Run before step 3.

In [5]:
## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

cluster mustafa7 does exist.
Status: CREATE_COMPLETE
Status: CREATE_COMPLETE
MasterServer: RUNNING
MasterServer: RUNNING
Output:"MasterPublicIP"="34.214.180.149"
Output:"MasterPrivateIP"="172.31.33.31"
Output:"GangliaPublicURL"="http://34.214.180.149/ganglia/"
Output:"GangliaPrivateURL"="http://172.31.33.31/ganglia/"

connecting
connected


## 3. Run the pipeline

This cell actually executes your pipeline. Make sure that steps 1 and 2 have been completed before running.

In [6]:
sample_list, group_list, pairs_list = DesignFileLoader.load_design_file(design_file)

PipelineManager.execute("ChiPSeq", ssh_client, project_name, workflow, analysis_steps, s3_input_files_address,
                       sample_list, group_list, s3_output_files_address, genome, style, pairs_list)

[['chipseq_sample1_chip.fastq.gz'], ['chipseq_sample1_input.fastq.gz'], ['chipseq_sample2_chip.fastq.gz'], ['chipseq_sample2_input.fastq.gz']]
['groupA', 'groupA', 'groupB', 'groupB']
{'chipseq_sample1_chip': 'chipseq_sample1_input', 'chipseq_sample2_chip': 'chipseq_sample2_input'}
making the yaml file...
copying yaml file to remote master node...
histone_output_check.yaml
/shared/workspace/Pipelines/yaml_files/ChiPSeq/homer
executing pipeline...
Executing nohup bash /shared/workspace/Pipelines/scripts/run.sh /shared/workspace/Pipelines/yaml_files/ChiPSeq/homer/histone_output_check.yaml /shared/workspace/logs/ChiPSeq/homer/histone_output_check ChiPSeq_homer


## 4. Check status of pipeline

This allows you to check the status of your pipeline. You can specify a step or set the step variable to "all". If you specify a step it should be one that is in your analysis_steps set. You can toggle how verbose the status checking is by setting the verbose flag (at the end of the second line) to False. 

In [ ]:
step = "all" #can be any step in analysis_steps or "all"
PipelineManager.check_status(ssh_client, step, "ChiPSeq", workflow, project_name, analysis_steps,verbose=True)

If your pipeline is finished run this cell just in case there's some processes still running.
This is only relevant if you plan on doing another run on the same cluster afterwards.

In [ ]:
PipelineManager.stop_pipeline(ssh_client)

## 5. Display MultiQC report

### Note: Run the cells below after the multiqc step is done

In [ ]:
# Download the multiqc html file to local
notebook_dir = os.getcwd().split("notebooks")[0] + "data/"
!aws s3 cp $s3_output_files_address/$project_name/$workflow/multiqc_report.html $notebook_dir

In [ ]:
from IPython.display import IFrame

IFrame(os.path.relpath("{}multiqc_report.html".format(notebook_dir)), width="100%", height=1000)